In [1]:
import pandas as pd
from taxi_demand_predictor.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73135,3.0,4.0,1.0,1.0,1.0,0.0,1.0,2.0,2.0,3.0,...,29.0,15.0,4.0,12.0,7.0,11.0,4.0,2022-10-27,265,12.0
73136,9.0,4.0,3.0,3.0,2.0,4.0,2.0,6.0,1.0,3.0,...,29.0,17.0,12.0,4.0,5.0,8.0,9.0,2022-10-28,265,3.0
73137,7.0,6.0,3.0,4.0,3.0,5.0,7.0,6.0,5.0,10.0,...,29.0,13.0,9.0,10.0,5.0,10.0,7.0,2022-10-29,265,6.0
73138,6.0,5.0,8.0,6.0,6.0,0.0,1.0,2.0,8.0,6.0,...,8.0,10.0,7.0,3.0,3.0,6.0,2.0,2022-10-30,265,10.0


In [2]:
from taxi_demand_predictor.data_split import train_test_split
from datetime import datetime
X_train, y_train, X_test, y_test = train_test_split(df, target_column_name='target_rides_next_hour', cutoff_date=datetime(2022, 6, 1, 0, 0, 0))


In [3]:
import lightgbm as lgb
past_ride_columns = [c for c in X_train.columns if c.startswith('rides_')]
X_train_only_numeric = X_train[past_ride_columns]

In [4]:
model = lgb.LGBMRegressor()
model.fit(X_train_only_numeric, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.045981 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 154010
[LightGBM] [Info] Number of data points in the train set: 32595, number of used features: 672
[LightGBM] [Info] Start training from score 11.571069


LGBMRegressor()

In [5]:
X_test_only_numeric = X_test[past_ride_columns]
predictions = model.predict(X_test_only_numeric)

from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=2.4709
